# Grab glacier centerline speeds and the plot the speed evolution time series

_Last modified by jukesliu@u.boisestate.edu on 2022-05-02._

In [1]:
import pandas as pd
import os
import numpy as np
from ordered_set import OrderedSet # pip install ordered-set
import cmocean
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
import datetime

from additional_functions import mytomd, unique_date_df, vector_magnitude

In [2]:
basepath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/' # SET WORKING DIRECTORY
# the root folder that holds centerline data

In [24]:
# grab reference centerline distances from a data file:
sorted_df = pd.read_csv(basepath+'custom_autoRIFT_cline_data/centerline_data_n.csv', # PATH TO THE SAR CSV
                    usecols=[2,3,4,5,6,7,8,9,10,11]) # may need to adjust column
dists = list(OrderedSet(sorted_df.dist_km))[1:]
dists.sort()
print(dists)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/custom_autoRIFT_cline_data/centerline_data_n.csv'

# 1A) Read in individual centerline profiles and combine

When speeds along centerline are stored in a folder containing a CSV file with the data for each date.

In [41]:
cfilespath = basepath+'custom_autoRIFT_cline_data/filtered-south/' # ENTER PATH TO THE FOLDER CONTAINING THE PROFILES

interp_dfs = []
for file in os.listdir(cfilespath):
    if file.startswith('profile') and file.endswith('.csv') and 'S' not in file:
        ds1 = file[8:16]; ds2 = file[17:25]
        profile_df = pd.read_csv(cfilespath+file, names=['dist_km','vmag_md']) # read in data
        
        # correct distance values if not correct
        maxdist = np.nanmax(profile_df.dist_km) # grab the maximum distance value 
        if  maxdist < 29.54666: # distance should go out to 29.54 km
            newdistkm = np.flip(np.array(profile_df.dist_km * 29.54666/maxdist)) # rescale to 29.54 km
            profile_df.dist_km = newdistkm # replace in dataframe
        
#         downsample to SAR dist values
        sample_indices = np.round(np.linspace(0,len(profile_df)-1,len(dists))) # grab sampling indexes for dists
        distances = dists
#         sample_indices = np.round(np.linspace(0,len(profile_df)-1,63)) # grab full profile
#         distances = np.flip(profile_df.dist_km[sample_indices])
        
        v_interp = np.flip(profile_df.vmag_md[sample_indices]) # grab down-sampled spped values
    
        # fill in ds1 and ds2 columns
        ds1s = np.full(np.size(v_interp),ds1) 
        ds2s = np.full(np.size(v_interp),ds2)

        # enter into dataframe
        interp_df = pd.DataFrame(list(zip(ds1s,ds2s,distances,v_interp)),columns=['ds1','ds2','dist_km','vmag'])
        
        # calculate datetimes
        interp_df['ds1'] = pd.to_datetime(ds1s, format='%Y%m%d')
        interp_df['ds2'] = pd.to_datetime(ds2s, format='%Y%m%d')
        interp_dfs.append(interp_df)

# enter into one dataframe
interp_total = pd.concat(interp_dfs).sort_values(by=['ds1','ds2','dist_km'])
interp_total = interp_total.drop_duplicates()
interp_total.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/custom_autoRIFT_cline_data/filtered-south/'

In [10]:
# NEW DATA
cfilespath = basepath+'/custom_autoRIFT_cline_data/filtered/' # ENTER PATH TO THE FOLDER CONTAINING THE PROFILES

interp_dfs = []
for file in os.listdir(cfilespath):
        ds1 = file.split('_')[0]; ds2 = file.split('_')[1]
        profile_df = pd.read_csv(cfilespath+file) # read in data
        
        v_interp = vector_magnitude(profile_df.vx, profile_df.vy)/365
        distances = profile_df.dist_m/1000
    
        # fill in ds1 and ds2 columns
        ds1s = np.full(np.size(v_interp),ds1) 
        ds2s = np.full(np.size(v_interp),ds2)

        # enter into dataframe
        interp_df = pd.DataFrame(list(zip(ds1s,ds2s,distances,v_interp)),columns=['ds1','ds2','dist_km','vmag'])
        
        # calculate datetimes
        interp_df['ds1'] = pd.to_datetime(ds1s, format='%Y%m%d')
        interp_df['ds2'] = pd.to_datetime(ds2s, format='%Y%m%d')
        interp_dfs.append(interp_df)

# enter into one dataframe
interp_total = pd.concat(interp_dfs).sort_values(by=['ds1','ds2','dist_km'])
interp_total = interp_total.drop_duplicates()
interp_total.head()

,ds1,ds2,dist_km,vmag
0,2013-04-14,2013-06-10,0.10,NaN
1,2013-04-14,2013-06-10,0.35,NaN
2,2013-04-14,2013-06-10,0.60,NaN
3,2013-04-14,2013-06-10,0.85,NaN
4,2013-04-14,2013-06-10,1.10,NaN


In [11]:
# grab the unique dates
df2 = unique_date_df(interp_total,'ds1','ds2')
df2

,ds1,ds2
0,2013-04-14,2013-06-10
1,2013-06-10,2013-06-17
2,2013-06-17,2013-07-12
3,2013-07-12,2013-07-28
4,2013-07-28,2013-08-13
...,...,...
200,2021-08-15,2021-08-30
201,2021-08-30,2021-09-07
202,2021-09-07,2021-09-19
203,2021-09-19,2021-09-22


In [12]:
# initialize speed grid
speed_grid = np.zeros((len(df2), len(distances)))

# fill in speed grid with distances
rown = 0
for idx, row in df2.iterrows():
    # grab the dates
    d1 = row.ds1; d2 = row.ds2
    
    # grab the part of the df matching those dates
    date_df = interp_total[interp_total.ds1 == d1]
    date_df = date_df[date_df.ds2 == d2]
    date_df.reset_index(drop=True, inplace=True)
    date_df = date_df.drop_duplicates(subset=['dist_km'],keep='last') # drop duplications
    
    # append into row of speed grid
    speed_grid[rown,:] = list(date_df.vmag) # add speed along centerline to speed_grid
    rown += 1
print(speed_grid.shape)

(205, 121)


### Fill in temporal gaps:

In [13]:
date_ends = np.array(list((zip(df2.ds1, df2.ds2)))).flatten() # intersperse ds1 and ds2

fill_dfs = []
counter = 0
# identify temporal gaps
for i in np.arange(0, len(date_ends),2):
    if i+2 < len(date_ends):
        # grab the two dates for that speed_grid()
        date1_start = date_ends[i]
        date1_end = date_ends[i+1]
        date2_start = date_ends[i+2]
        print(str(date1_start)[0:10],str(date1_end)[0:10]) # print the date start and end
    
        if not date1_end == date2_start: # if the end date and the next start date don't match
            print()
            print('Gap between', date1_end, 'and', date2_start) # Gap found
            print()

            # OPTION 1: fill in all gaps gap with Nans
            ds1s = np.full(np.size(distances),date1_end) 
            ds2s = np.full(np.size(distances),date2_start)
            nans = np.empty(len(distances)); nans[:] = np.nan # create list of nans to fill
            fill_df = pd.DataFrame(list(zip(ds1s, ds2s,distances,nans)),columns=interp_total.columns)
            fill_dfs.append(fill_df)
            
#             # OPTION 2: fill in the gaps with SAR data
#             sorted_df = pd.read_csv(basepath+'ASF_autoRIFT/centerline_data_n.csv', # PATH TO THE SAR CSV
#                                     usecols=[2,3,4,5,6,7,8,9,10,11]) # may need to adjust column
#             # convert dates to datetime objects using pd
#             sorted_df['mid_date'] = pd.to_datetime(sorted_df.mid_date, format='%Y%m%d')
#             sorted_df['ds1'] = pd.to_datetime(sorted_df.ds1, format='%Y-%m-%d')
#             sorted_df['ds2'] = pd.to_datetime(sorted_df.ds2, format='%Y-%m-%d')
#             sorted_df['vmag'] = mytomd(sorted_df.vmag) # convert velocities to m/day
#             sorted_df['v_error'] = mytomd(sorted_df.v_error)
#             sorted_df = sorted_df.drop_duplicates(subset=['ds1','lat'],keep='first') # drop overlapping dates

#             # grab each unique date pair
#             dates_df = unique_date_df(sorted_df,'ds1','ds2')
#             gap_df = dates_df[(dates_df.ds1 >= date1_end) & (dates_df.ds2 <= date2_start)] # find those in gap
#             if len(gap_df) > 0: # fill in gaps with SAR data
#                 print('Fill with:')
#                 for idx, row in gap_df.iterrows():
#                     print(row.ds1, row.ds2)
#                     df = sorted_df[(sorted_df.ds1 == row.ds1) & (sorted_df.ds2 == row.ds2)]
#                     fill_df = df[["ds1", "ds2","dist_km", "vmag"]]
#                     fill_dfs.append(fill_df)
#             else: # if no optical data found, fill in with nans
#                 print("Fill with nans")
#                 ds1s = np.full(np.size(dists),date1_end) 
#                 ds2s = np.full(np.size(dists),date2_start)
#                 nans = np.empty(len(dists)); nans[:] = np.nan # create list of nans to fill
#                 fill_df = pd.DataFrame(list(zip(ds1s, ds2s, dists,nans)),columns=interp_total.columns)
#                 fill_dfs.append(fill_df)
                
                
            counter+=1  # count the gaps      
print(counter, 'gaps')

# add gap-filling data back into the dataframe
interp_total_filled = pd.concat([interp_total,pd.concat(fill_dfs)]).sort_values(by=['ds1','ds2','dist_km'])

2013-04-14 2013-06-10
2013-06-10 2013-06-17
2013-06-17 2013-07-12
2013-07-12 2013-07-28
2013-07-28 2013-08-13
2013-08-13 2013-10-07

Gap between 2013-10-07 00:00:00 and 2014-02-28 00:00:00

2014-02-28 2014-03-25
2014-03-25 2014-04-01
2014-04-01 2014-04-10
2014-04-10 2014-04-26
2014-04-26 2014-05-03
2014-05-03 2014-06-04
2014-06-04 2014-07-22
2014-07-22 2014-07-31
2014-07-31 2014-08-23
2014-08-23 2014-09-01
2014-09-01 2014-09-24

Gap between 2014-09-24 00:00:00 and 2015-03-28 00:00:00

2015-03-28 2015-04-13
2015-04-13 2015-05-06
2015-05-06 2015-05-15
2015-05-15 2015-05-22
2015-05-22 2015-05-31
2015-05-31 2015-06-23
2015-06-23 2015-07-09
2015-07-09 2015-07-25
2015-07-25 2015-08-03

Gap between 2015-08-03 00:00:00 and 2016-02-18 00:00:00

2016-02-18 2016-03-14
2016-03-14 2016-03-21
2016-03-21 2016-05-08
2016-05-08 2016-06-09
2016-06-09 2016-07-11
2016-07-11 2016-08-28
2016-08-28 2016-09-29
2016-09-29 2016-10-08
2016-10-08 2016-10-24

Gap between 2016-10-24 00:00:00 and 2017-02-05 00:00:00

# 1B) Read in combined centerline data file

When speeds along centerline for all dates have been combined into one large file.

In [28]:
# csvpath = basepath+'ASF_autoRIFT/centerline_data_n.csv' # ENTER PATH TO THE CSV FILE WITH ALL DATA COMBINED
# csvpath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/its_live/notebooks/'
csvpath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/ASF_autoRIFT/'
csvpath += 'centerline_data_s.csv'

# read into a dataframe
sorted_df = pd.read_csv(csvpath, usecols=[2,3,4,6,7,8,10,11]) # ADJUST COLUMNS AS NEEDED, DROP MID_DATE
sorted_df.head()

,lon,lat,PSx,vmag,v_error,dist_km,ds1,ds2
0,-139.942202,60.119742,-3.296994e+06,132.0,54.0,0.000000,2016-10-20,2016-11-13
1,-139.937770,60.120118,-3.296972e+06,62.0,46.0,0.249907,2016-10-20,2016-11-13
2,-139.933279,60.120221,-3.296983e+06,23.0,57.0,0.499879,2016-10-20,2016-11-13
3,-139.928796,60.120056,-3.297024e+06,78.0,61.0,0.749845,2016-10-20,2016-11-13
4,-139.924315,60.119860,-3.297069e+06,65.0,51.0,0.999845,2016-10-20,2016-11-13


In [29]:
# convert dates to datetime objects using pd
sorted_df['ds1'] = pd.to_datetime(sorted_df.ds1, format='%Y-%m-%d')
sorted_df['ds2'] = pd.to_datetime(sorted_df.ds2, format='%Y-%m-%d')

# convert velocities to m/day
sorted_df['vmag'] = mytomd(sorted_df.vmag)
sorted_df['v_error'] = mytomd(sorted_df.v_error)

# drop overlapping dates!
sorted_df = sorted_df.drop_duplicates(subset=['ds1','lat'],keep='first') 
sorted_df.head()

,lon,lat,PSx,vmag,v_error,dist_km,ds1,ds2
0,-139.942202,60.119742,-3.296994e+06,0.361644,0.147945,0.000000,2016-10-20,2016-11-13
1,-139.937770,60.120118,-3.296972e+06,0.169863,0.126027,0.249907,2016-10-20,2016-11-13
2,-139.933279,60.120221,-3.296983e+06,0.063014,0.156164,0.499879,2016-10-20,2016-11-13
3,-139.928796,60.120056,-3.297024e+06,0.213699,0.167123,0.749845,2016-10-20,2016-11-13
4,-139.924315,60.119860,-3.297069e+06,0.178082,0.139726,0.999845,2016-10-20,2016-11-13


In [30]:
# grab each unique date pair
df2 = unique_date_df(sorted_df,'ds1','ds2')
df2.head()

,ds1,ds2
0,2016-10-20,2016-11-13
1,2016-11-13,2016-12-07
2,2016-12-07,2016-12-31
3,2016-12-31,2017-01-24
4,2017-01-24,2017-02-17


In [34]:
# initialize speed grid
# speed_grid = np.zeros((len(df2), len(dists)))
speed_grid = np.zeros((len(df2), len(list(set(sorted_df.dist_km)))))
print(speed_grid.shape)

# fill in speed grid with speed values for each date pair
rown = 0
for idx, row in df2.iterrows():
    # grab the dates
    d1 = row.ds1; d2 = row.ds2
    
    # grab the part of the df matching those dates
    date_df = sorted_df[sorted_df.ds1 == d1]
    date_df = date_df[date_df.ds2 == d2]
    date_df.reset_index(drop=True, inplace=True)
    date_df = date_df.drop_duplicates(subset='dist_km',keep='first')
#     print(len(date_df))
#     if len(date_df) < 218:
#         print(np.array(date_df.dist_km))

    # append into row of speed grid
    speed_grid[rown,:] = list(date_df.vmag) # add speed along centerline to speed_grid   
    rown += 1

(97, 98)


In [35]:
# np.array(date_df.dist_km)

### Fill in temporal gaps:

In [36]:
# intersperse ds1 and ds2
date_ends = np.array(list((zip(df2.ds1, df2.ds2)))).flatten()

fill_dfs = []
counter = 0
# identify number of gaps in to fill in data
for i in np.arange(0, len(date_ends),2):
    if i+2 < len(date_ends):
        # grab the two dates for that speed_grid()
        date1_start = date_ends[i]
        date1_end = date_ends[i+1]
        date2_start = date_ends[i+2]
        print(str(date1_start)[0:10],str(date1_end)[0:10]) # print the date start and end
        
        if not date1_end == date2_start: # if the end date and the next start date don't match
            print()
            print('Gap between', str(date1_end)[0:10], 'and', str(date2_start)[0:10])
            print()
            
#             # OPTION 1: fill in gaps with Nans
#             ds1s = np.full(np.size(dists),date1_end) 
#             ds2s = np.full(np.size(dists),date2_start)
#             nans = np.empty(len(dists)); nans[:] = np.nan # create list of nans to fill
#             fill_df = pd.DataFrame(list(zip(nans, nans, nans, nans, nans, nans, dists, nans, ds1s, ds2s)),
#                                    columns=sorted_df.columns)
#             fill_dfs.append(fill_df)
            
            # OPTION 2: fill in gaps with optical data
            for file in os.listdir(cfilespath): # path to optical data
                if file.startswith('profile') and file.endswith('.csv') and 'S' not in file:
                    dstr1 = file[8:16]; dstr2 = file[17:25]
                    ds1 = pd.to_datetime(dstr1,format='%Y%m%d')
                    ds2 = pd.to_datetime(dstr2,format='%Y%m%d')
                    
                    # find optical data between the gap dates
                    if (ds1 >= date1_end) and (ds2 <= date2_start): 
                        print('Fill with optical data :',dstr1, dstr2)
                        profile_df = pd.read_csv(cfilespath+file, names=['dist_km','vmag_md'])
                        
                        # down-sample the optical data
                        maxdist = np.nanmax(profile_df.dist_km)
                        if  maxdist < 29.54666: # distance should go out to 29.54 km
                            newdistkm = np.flip(np.array(profile_df.dist_km * 29.54666/maxdist)) # rescale to 29.54 km
                            profile_df.dist_km = newdistkm # replace in dataframe
                        sample_indices = np.round(np.linspace(0,len(profile_df)-1,40))
                        v_interp = np.flip(profile_df.vmag_md[sample_indices])

                        # fill in a ds1 and ds2 columns
                        ds1s = np.full(np.size(v_interp),ds1) 
                        ds2s = np.full(np.size(v_interp),ds2)
                        nans = np.empty(np.size(v_interp)); nans[:] = np.nan # create list of nans to fill

                        # enter into dataframe
                        fill_df = pd.DataFrame(list(zip(nans, nans, nans, nans, v_interp, nans, dists, nans, ds1s, ds2s)),
                                           columns=sorted_df.columns)
                        fill_dfs.append(fill_df)
            counter+=1
print(counter, 'gaps')

# return the filled df
interp_total_filled = pd.concat([sorted_df,pd.concat(fill_dfs)]).sort_values(by=['ds1','ds2','dist_km'])
interp_total_filled.head()

2016-10-20 2016-11-13
2016-11-13 2016-12-07
2016-12-07 2016-12-31
2016-12-31 2017-01-24
2017-01-24 2017-02-17
2017-02-17 2017-03-01
2017-03-01 2017-03-13

Gap between 2017-03-13 and 2017-05-18



NameError: name 'cfilespath' is not defined

In [ ]:
# FILL ALL REMAINING GAPS IF OPTICAL DATA ARE INPUT
df2 = pd.DataFrame(list(OrderedSet(zip(interp_total_filled.ds1, interp_total_filled.ds2))),
                   columns=['ds1','ds2'])
date_ends = np.array(list((zip(df2.ds1, df2.ds2)))).flatten()

fill_dfs = []
counter = 0
# identify number of gaps in to fill in data
for i in np.arange(0, len(date_ends),2):
    if i+2 < len(date_ends):
        # grab the two dates for that speed_grid()
        date1_start = date_ends[i]
        date1_end = date_ends[i+1]
        date2_start = date_ends[i+2]
        print(date1_start, date1_end)
        
        if not date1_end == date2_start: # if the end date and the next start date don't match
            print()
            print('Gap between', date1_end, 'and', date2_start)
            print()
            
            # fill in a the gaps with nans
            ds1s = np.full(np.size(dists),date1_end) 
            ds2s = np.full(np.size(dists),date2_start)
            nans = np.empty(len(dists)); nans[:] = np.nan # create list of nans to fill
            fill_df = pd.DataFrame(list(zip(nans, nans, nans, nans, nans, nans, dists, nans, ds1s, ds2s)),
                                   columns=sorted_df.columns)
            fill_dfs.append(fill_df)
            counter+=1  
print(counter, 'gaps')

interp_total_filled = pd.concat([interp_total_filled,pd.concat(fill_dfs)]).sort_values(by=['ds1','ds2','dist_km'])
interp_total_filled.head()

## 2) Plot speed evolution time series

In [14]:
# grab ordered dates as a dataeframe
df2_filled = unique_date_df(interp_total_filled,'ds1','ds2')
df2_filled

,ds1,ds2
0,2013-04-14,2013-06-10
1,2013-06-10,2013-06-17
2,2013-06-17,2013-07-12
3,2013-07-12,2013-07-28
4,2013-07-28,2013-08-13
...,...,...
272,2021-08-15,2021-08-30
273,2021-08-30,2021-09-07
274,2021-09-07,2021-09-19
275,2021-09-19,2021-09-22


In [18]:
# Re-create speed grid with gap-filled data
speed_grid_filled = np.zeros((len(df2_filled), len(distances)))

# speed grid with gap-filled data
rown = 0
for idx, row in df2_filled.iterrows():
    d1 = row.ds1; d2 = row.ds2 # grab the dates
    
    # grab the part of the df matching those dates
    date_df = interp_total_filled[interp_total_filled.ds1 == d1]
    date_df = date_df[date_df.ds2 == d2]
    date_df.reset_index(drop=True, inplace=True)
    date_df = date_df.drop_duplicates(subset=['dist_km'],keep='last') # drop duplications
    
    # append into row of speed grid
    speed_grid_filled[rown,:] = list(date_df.vmag) # add speed along centerline to speed_grid
    rown += 1
print(speed_grid_filled.shape)

(277, 121)


In [16]:
# determine the grid spacing based on the time differences
spacing = np.round(np.diff(df2_filled.ds1)/np.min(np.diff(df2_filled.ds1)))
spacing = np.append(spacing, spacing[-1]) # add last spacing
spacing

/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_43039/2696162903.py:2: RuntimeWarning: divide by zero encountered in true_divide
  spacing = np.round(np.diff(df2_filled.ds1)/np.min(np.diff(df2_filled.ds1)))
/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_43039/2696162903.py:2: RuntimeWarning: invalid value encountered in true_divide
  spacing = np.round(np.diff(df2_filled.ds1)/np.min(np.diff(df2_filled.ds1)))


array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, nan, inf, nan, inf, inf, nan, inf, inf, inf, nan, inf,
       nan, inf, nan, inf, nan, inf, inf, inf, nan, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, nan, inf,
       nan, inf, nan, inf, nan, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, nan, inf, nan, inf, nan, inf, nan, inf, inf, nan, inf,
       inf, nan, nan, nan, inf, nan, inf, nan, inf, nan, inf, inf, nan,
       inf, inf, nan, inf, nan, nan, inf, nan, inf, inf, inf, inf, inf,
       nan, inf, nan, inf, nan, inf, nan, inf, inf, inf, inf, inf, inf,
       inf, nan, inf, nan, inf, nan, inf, nan, inf, nan, inf, nan, inf,
       nan, nan, inf, nan, inf, nan, nan, inf, nan, inf, inf, in

In [17]:
# create spaced grid
speed_spaced = np.repeat(speed_grid_filled,spacing.astype(int),0) # along rows
# speed_spaced[speed_spaced == 0] = np.NaN
speed_spaced

ValueError: repeats may not contain negative values.

# Plot:

In [ ]:
# User input to properly generate axis labels for the plot:
# if grid tick spacing is unknown, run the cell below with the tick labels turned off to see tick spacing

# create centerline (x-axis) labels 
x = np.linspace(dists[0], dists[-1], speed_grid.shape[1]+1)
x_labels = np.insert(x[::10],0,0).astype(int) # INPUT GRID TICK SPACING FOR X (e.g. 10 or 5)
print(x_labels)

# create y-axis labels
t = pd.date_range(start='2013-06-10',end='2021-09-22',periods=len(speed_spaced)) # INPUT START AND END DATE
y = np.insert(t[::500],0,t[0]) # INPUT GRID TICK SPACING (e.g. 250, 200, 500, or 20)
y_labels = [ystr[:7] for ystr in y.astype(str)] # grab the first 7 digits YYYY-MM for each timestamp in y
print(y_labels)

In [ ]:
# Plot the properly-spaced speed evolution map with imshow
fig,ax = plt.subplots(1,1,figsize=(6,6))
ax.set_facecolor('black')
grid = plt.imshow(speed_spaced,aspect=0.03, # 0.03 for full 2013-2022
                  cmap=cmocean.cm.thermal,
#                  vmin=0, vmax=7) # linear coloring
                  norm=LogNorm(vmin=1, vmax=25)) # log norm coloring
fig.colorbar(grid, orientation="vertical",label="Surface speed (m/day)")

# set tick labels
ax.set_xticklabels(x_labels); plt.xlabel('Distance from terminus (km)')
ax.set_yticklabels(y_labels)

# plt.savefig(basepath+"figures/Optical_filled_with_SAR.png",dpi=300) # save figure
# plt.title('SAR and Optical') # add title
plt.show()

In [21]:
interp_df

,ds1,ds2,dist_km,vmag
0,2020-04-27,2020-05-10,0.10,NaN
1,2020-04-27,2020-05-10,0.35,NaN
2,2020-04-27,2020-05-10,0.60,NaN
3,2020-04-27,2020-05-10,0.85,NaN
4,2020-04-27,2020-05-10,1.10,NaN
...,...,...,...,...
116,2020-04-27,2020-05-10,29.10,NaN
117,2020-04-27,2020-05-10,29.35,NaN
118,2020-04-27,2020-05-10,29.60,NaN
119,2020-04-27,2020-05-10,29.85,NaN
